In [943]:
#Run this file to clean up the source csv docs (before running the SQL database).

import os
import sqlite3
import zipfile
import numpy as np
import pandas as pd
from fractions import Fraction

#pandas reading in the files
goodreadsraw = pd.read_csv("data/goodreads-rawdata.csv.zip", compression="zip", index_col=0)
loulibraw = pd.read_csv("data/loumetrolib-rawdata.csv.zip", compression="zip")



#GOODREADS DATA CLEANING
#dropping duplicate records
goodreads = goodreadsraw.drop_duplicates(subset=["isbn", "title", "authors", "url"], keep = "first")
goodreads = goodreadsraw.drop_duplicates(subset=["url"], keep = "first")
#dropping irrelevant columns
goodreads = goodreads.drop(["language", "description", "series_title", "series_release_number", "publisher", "num_pages", "url"], axis=1)
  #replacing null values with "Not Provided"
  #goodreads = goodreads.fillna("Not provided")
  #replacing invalid values with ""
  #goodreads.loc[goodreads["isbn"].str.contains("ISBN10:"), "isbn"] = ""
goodreads = goodreads.replace(goodreads["isbn"].str.contains("ISBN10:"), np.nan)
#slicing goodreads publishing dates into just years
goodreads["publication_date"] = goodreads["publication_date"].str.slice(start=-4)
#Reset the index
goodreads = goodreads.reset_index(drop=True)
#reordering columns
goodreads = goodreads[["isbn", "title", "authors", "publication_date", "format", "genres",
"rating_score", "num_ratings", "num_reviews", "current_readers", "want_to_read", "price"]]
#removing ebooks, keeping books & audiobooks
indexkindle = goodreads[(goodreads["format"] == "Kindle Edition")].index
goodreads.drop(indexkindle , inplace=True)
indexebook = goodreads[(goodreads["format"] == "ebook")].index
goodreads.drop(indexebook , inplace=True)
indexnook = goodreads[(goodreads["format"] == "Nook")].index
goodreads.drop(indexnook , inplace=True)
indexcass = goodreads[(goodreads["format"] == "Audio Cassette")].index
goodreads.drop(indexcass , inplace=True)
indexcd = goodreads[(goodreads["format"] == "Audio CD")].index
goodreads.drop(indexcd , inplace=True)


#LIBRARY DATA CLEANING
#Sort by library"s "unique identifier of a bibliographic record" called BibNum

loulib = loulibraw.sort_values(by=["BibNum"], ascending=True)
#dropping irrelevant columns
loulib = loulib.drop(["ItemLocation", "ReportDate"], axis=1)
#dropping 2024 publications (to match publication date range of goodreads data)
indexPub = loulib[(loulib["PublicationYear"] == 2024)].index
loulib.drop(indexPub , inplace=True)
#dropping laptops & listening devices
indexLaptop = loulib[(loulib["ItemCollection"] == "Laptop")].index
loulib.drop(indexLaptop , inplace=True)
indexDevice = loulib[(loulib["ItemCollection"] == "Listening Device")].index
loulib.drop(indexDevice , inplace=True)
#dropping electronic books, keeping books & audiobooks
indexelec = loulib[(loulib["ItemCollection"] == "Electronic")].index
loulib.drop(indexelec , inplace=True)
indexLaptop = loulib[(loulib["ItemCollection"] == "Laptop")].index
loulib.drop(indexLaptop , inplace=True)
indexserial = loulib[(loulib["ItemType"] == "Serial")].index
loulib.drop(indexserial , inplace=True)
indexelecr = loulib[(loulib["ItemType"] == "Electronic Resources")].index
loulib.drop(indexelecr , inplace=True)
indexmusiccd = loulib[(loulib["ItemType"] == "Music CD")].index
loulib.drop(indexmusiccd , inplace=True)
indexmusicsr = loulib[(loulib["ItemType"] == "Musical Sound Recording")].index
loulib.drop(indexmusicsr , inplace=True)
indexsr = loulib[(loulib["ItemType"] == "Sound Recording")].index
loulib.drop(indexsr , inplace=True)
indexvideotp = loulib[(loulib["ItemType"] == "Videotape")].index
loulib.drop(indexvideotp , inplace=True)
indexbluray = loulib[(loulib["ItemType"] == "Blu-ray Disk")].index
loulib.drop(indexbluray , inplace=True)
indexprojm = loulib[(loulib["ItemType"] == "Projected Medium")].index
loulib.drop(indexprojm , inplace=True)
indexebk = loulib[(loulib["ItemType"] == "Ebook")].index
loulib.drop(indexebk , inplace=True)
indexblurayd = loulib[(loulib["ItemType"] == "Blu-ray + DVD")].index
loulib.drop(indexblurayd , inplace=True)

#reformatting author names
loulib["Author"] = loulib["Author"].str.replace(", \d+\D\d+", "")
loulib["Author"] = loulib["Author"].str.replace(", \d+-", "")
loulib["Author"] = loulib["Author"].str.replace(".", "")
loulib["Author"] = loulib["Author"].str.split(",").str[::-1].str.join(" ").str.strip()

#Creating a pivottable that counts library copies, column names: BibNum and Copies
loulibcount = loulib.pivot_table(index=["BibNum"], aggfunc={"BibNum": "count"})
# Rename the BibNum Count column to Copies
loulibcount = loulibcount.rename(columns={"BibNum": "Copies"})

#Add a column to identify multiple versions of the same book
loulib["TitleAuth"] = loulib["Title"].astype(str) + loulib["Author"]

#dropping duplicate Library records
loulib = loulib.drop_duplicates(subset=["BibNum"], keep = "first")

#Creating a pivottable that counts library book Versions/Editions, column names: TitleAuth and Versions
loulibversions = loulib.pivot_table(index=["TitleAuth"], aggfunc={"TitleAuth": "count"})
# Rename the TitleAuth Count column to Versions
loulibversions = loulibversions.rename(columns={"TitleAuth": "Versions"})

#dropping duplicate Library records
loulib = loulib.drop_duplicates(subset=["TitleAuth"], keep = "first")

#Adding Copies column to library data
loulib = pd.merge(loulib, loulibcount, left_on="BibNum", right_on="BibNum", how="left")
#Adding Versions column to library data
loulib = pd.merge(loulib, loulibversions, left_on="TitleAuth", right_on="TitleAuth", how="left")

#correcting null Version values
loulib["Versions"] = loulib["Versions"].fillna(1)
#reordering columns
loulib = loulib[[ "ISBN", "Title", "Author", "PublicationYear", "ItemType", "ItemCollection", "ItemPrice", "Copies", "Versions"]]

#making invalid years NAN
loulib.loc[loulib["PublicationYear"] > 2024] = np.nan
loulib.loc[loulib["PublicationYear"] < 1000] = np.nan
#making ISBN & BibNum strings
loulib["ISBN"] = loulib["ISBN"].astype(object)
#df = df.astype({"B": int, "C": int})
#loulib["BibNum"] = obj(loulib["BibNum"])



#PREPARING FOR MERGE
#rename columns for indexing
loulib.rename(columns={"ISBN": "isbn"}, inplace=True)
loulib.rename(columns={"Title": "title"}, inplace=True)
loulib.rename(columns={"Author": "authors"}, inplace=True)
loulib = loulib.drop(["PublicationYear"], axis=1)
#reformatting author names
goodreads["authors"] = goodreads["authors"].str.replace(".", "")


loulib = pd.DataFrame(loulib)
goodreads = pd.DataFrame(goodreads)

#showing the library data before the merge
loulib

C:\Users\Aliyah Gant\AppData\Local\Temp\ipykernel_15520\4025312692.py:88: FutureWarning: The default value of regex will change from True to False in a future version.
  loulib["Author"] = loulib["Author"].str.replace(", \d+\D\d+", "")
C:\Users\Aliyah Gant\AppData\Local\Temp\ipykernel_15520\4025312692.py:89: FutureWarning: The default value of regex will change from True to False in a future version.
  loulib["Author"] = loulib["Author"].str.replace(", \d+-", "")
C:\Users\Aliyah Gant\AppData\Local\Temp\ipykernel_15520\4025312692.py:90: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  loulib["Author"] = loulib["Author"].str.replace(".", "")
C:\Users\Aliyah Gant\AppData\Local\Temp\ipykernel_15520\4025312692.py:139: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single characte

,isbn,title,authors,ItemType,ItemCollection,ItemPrice,Copies,Versions
0,9780670300440.0,An exaltation of larks : the ultimate edition,James Lipton,Book,Adult Non-Fiction,16.00,3.0,1.0
1,9780004133300.0,Collins drawing for beginners : a step-by-step...,NaN,Book,Adult Non-Fiction,24.95,1.0,1.0
2,9780936070209.0,The septic system owner's manual,Lloyd Kahn,Book,Adult Non-Fiction,17.95,1.0,2.0
3,9780062515841.0,The invitation,Oriah Mountain Dreamer,Book,Adult Non-Fiction,21.95,1.0,1.0
4,9781840466713.0,Introducing Romanticism,Duncan Heath,Book,Adult Non-Fiction,12.95,8.0,1.0
...,...,...,...,...,...,...,...,...
427453,9780268036140.0,The 1916 irish rebellion,Bríona Nic Dhiarmada,Book,Adult Non-Fiction,39.60,1.0,1.0
427454,9781442244610.0,The 1916 rising: the photographic record,Turtle Bunbury,Book,Adult Non-Fiction,30.76,1.0,1.0
427455,9798988486114.0,Captured!: a world war ii memoir,Hugh O'Neill,Book,Adult Non-Fiction,24.99,1.0,1.0
427456,9780300272680.0,Shadows at noon: the south asian twentieth cen...,Joya Chatterji,Book,Adult Non-Fiction,35.20,1.0,1.0


In [944]:
#INNER MERGE & create clean data csv file
clean = pd.merge(loulib, goodreads, on=["title", "authors"], suffixes=("_lib", "_gr"), how="inner")
clean = clean.sort_values(by=["title"], ascending=True)
clean = clean.drop(["isbn_lib", "ItemType", "ItemCollection", "ItemPrice", "isbn_gr", "format", "price"], axis=1)
clean.to_csv("data/clean/booksclean.csv", index=False)
clean

,title,authors,Copies,Versions,publication_date,genres,rating_score,num_ratings,num_reviews,current_readers,want_to_read
101,11/22/63,Stephen King,3.0,1.0,2011,"['Fiction', 'Historical Fiction', 'Science Fic...",4.33,533469.0,47249.0,31000.0,25100.0
40,Affair,Amanda Quick,1.0,1.0,1985,"['Romance', 'Historical Romance', 'Historical'...",3.91,7193.0,290.0,511.0,1683.0
4,Airframe,Michael Crichton,5.0,1.0,1996,"['Fiction', 'Thriller', 'Mystery', 'Science Fi...",3.73,83882.0,2476.0,1864.0,25900.0
147,Alexander Hamilton,Ron Chernow,3.0,2.0,2004,"['History', 'Biography', 'Nonfiction', 'Americ...",4.22,179547.0,10065.0,34400.0,309000.0
201,Alias Grace,Margaret Atwood,6.0,2.0,1996,"['Historical Fiction', 'Fiction', 'Mystery', '...",4.04,142363.0,8673.0,8659.0,149000.0
...,...,...,...,...,...,...,...,...,...,...,...
110,"Will Grayson, Will Grayson",John Green,1.0,3.0,2010,"['Young Adult', 'Contemporary', 'Fiction', 'LG...",3.70,396219.0,20553.0,7660.0,243.0
92,Wintergirls,Laurie Halse Anderson,4.0,3.0,2009,"['Young Adult', 'Fiction', 'Contemporary', 'Me...",3.96,120111.0,10077.0,2267.0,106000.0
143,Wyrms,Orson Scott Card,2.0,1.0,1987,"['Science Fiction', 'Fantasy', 'Fiction', 'Sci...",3.40,5936.0,305.0,53.0,1456.0
109,Zeitoun,Dave Eggers,1.0,2.0,2009,"['Nonfiction', 'Biography', 'History', 'Memoir...",4.05,78157.0,8051.0,1226.0,38700.0


In [945]:
#How many unique physical books are in the Goodreads data?

print("Total number of physical books in the Goodreads data: ", goodreads.shape[0])
#How many unique physical books are in the library system?
print("Total number of physical books in the Louisville library system: ", loulib.shape[0])
#How many total book COPIES are in the library system?
llcs = loulibcount.sum().astype(str)
print("Total number of physical library book copies: ", llcs.str.cat(sep="\n"))
#How many total book EDITIONS are in the library system?
llbv = loulibversions.sum().astype(str)
print("Total number of physical library book editions: ", llbv.str.cat(sep="\n"))


print("How many physical books are listed on Goodreads AND in the Louisville library system?: ", clean.shape[0])

Total number of physical books in the Goodreads data:  3752
Total number of physical books in the Louisville library system:  427458
Total number of physical library book copies:  1335172
Total number of physical library book editions:  461196
How many physical books are listed on Goodreads AND in the Louisville library system?:  255


In [946]:
clean.describe()

,Copies,Versions,rating_score,num_ratings,num_reviews,current_readers,want_to_read
count,255.000000,255.000000,255.000000,2.550000e+02,255.000000,255.000000,254.000000
mean,4.219608,1.596078,3.946706,1.481808e+05,9974.435294,6270.313725,107652.173228
std,5.699738,0.863513,0.247027,2.459435e+05,17489.289433,12755.505105,168403.103985
min,1.000000,1.000000,3.220000,1.065000e+03,106.000000,18.000000,243.000000
25%,1.000000,1.000000,3.785000,1.644700e+04,936.500000,448.000000,7133.000000
50%,2.000000,1.000000,3.970000,5.218400e+04,3543.000000,1721.000000,33550.000000
75%,5.000000,2.000000,4.120000,1.743830e+05,10891.500000,5475.500000,132750.000000
max,43.000000,5.000000,4.520000,1.610506e+06,171602.000000,102000.000000,1000000.000000


In [947]:
print("What the above Summary chart of the clean data tells us:")
print("The average amount of physical library book copies listed on Goodreads is 4.")
print("The maximum amount of physical library book copies listed on Goodreads is 43.")
print("The lowest, average, and highest ratings for physical library books listed on Goodreads: 3.22, 3.95, 4.52")
print("The lowest, average, and highest number of ratings for physical library books listed on Goodreads: 1065, 148180, 1610506")
print("The lowest, average, and highest number of reviews for physical library books listed on Goodreads: 106, 9974, 171602")
print("The lowest, average, and highest number of current readers for physical library books listed on Goodreads: 18, 6270, 102000")
print("The lowest, average, and highest number of users who want to read a specific book for physical library books listed on Goodreads: 243, 107652, 1Million")

print("The maximum amount of physical library book copies is 43.")

What the above Summary chart of the clean data tells us:
The average amount of physical library book copies listed on Goodreads is 4.
The maximum amount of physical library book copies listed on Goodreads is 43.
The lowest, average, and highest ratings for physical library books listed on Goodreads: 3.22, 3.95, 4.52
The lowest, average, and highest number of ratings for physical library books listed on Goodreads: 1065, 148180, 1610506
The lowest, average, and highest number of reviews for physical library books listed on Goodreads: 106, 9974, 171602
The lowest, average, and highest number of current readers for physical library books listed on Goodreads: 18, 6270, 102000
The lowest, average, and highest number of users who want to read a specific book for physical library books listed on Goodreads: 243, 107652, 1Million
The maximum amount of physical library book copies is 43.


In [948]:
print("Are there correlations in the data?")
clean.corr()

Are there correlations in the data?


,Copies,Versions,rating_score,num_ratings,num_reviews,current_readers,want_to_read
Copies,1.000000,-0.020302,-0.007258,0.018797,0.053027,0.108882,0.073335
Versions,-0.020302,1.000000,0.081222,0.395257,0.169658,0.133563,0.300835
rating_score,-0.007258,0.081222,1.000000,0.286458,0.297449,0.331476,0.309981
num_ratings,0.018797,0.395257,0.286458,1.000000,0.830881,0.706573,0.833099
num_reviews,0.053027,0.169658,0.297449,0.830881,1.000000,0.796176,0.854149
current_readers,0.108882,0.133563,0.331476,0.706573,0.796176,1.000000,0.752834
want_to_read,0.073335,0.300835,0.309981,0.833099,0.854149,0.752834,1.000000


In [949]:
print("What the correlations show: ")
print("As we look at Goodreads books that have a higher # of copies in stock at the library, the quantity of book editions available decreases.")
print("As we look at Goodreads books that have a higher # of copies in stock at the library, the book ratings worsen.")
print("As we look at books that have more copies in stock at the library, the # of Goodreads users who want to read the book increases.")
print("As we look at books with more editions available at the library, the quantity of Goodreads ratings and users who want to read the book increase.")
print("As we look at a higher number of ratings, the number of reviews, current readers, and folks who want to read the specific book all increase significantly per Goodreads.")

What the correlations show: 
As we look at Goodreads books that have a higher # of copies in stock at the library, the quantity of book editions available decreases.
As we look at Goodreads books that have a higher # of copies in stock at the library, the book ratings worsen.
As we look at books that have more copies in stock at the library, the # of Goodreads users who want to read the book increases.
As we look at books with more editions available at the library, the quantity of Goodreads ratings and users who want to read the book increase.
As we look at a higher number of ratings, the number of reviews, current readers, and folks who want to read the specific book all increase significantly per Goodreads.


In [950]:
print("Which books does the Louisville library have out of the top 500 rated physical Goodreads books?")

goodreadst = goodreads.sort_values(by=["rating_score"], ascending=False)
goodreadst = goodreadst.head(500)

toprated = pd.merge(loulib, goodreadst, on=["title", "authors"], suffixes=("_lib", "_gr"), how="inner")
toprated[["title", "authors", "publication_date", "rating_score"]]

Which books does the Louisville library have out of the top 500 rated physical Goodreads books?


,title,authors,publication_date,rating_score
0,Matilda,Roald Dahl,1988,4.34
1,The Polar Express,Chris Van Allsburg,1985,4.32
2,Wilfrid Gordon McDonald Partridge,Mem Fox,1984,4.42
3,Columbine,Dave Cullen,2009,4.29
4,11/22/63,Stephen King,2011,4.33
5,Still Alice,Lisa Genova,2007,4.32
6,Homegoing,Yaa Gyasi,2016,4.47
7,Watchmen,Alan Moore,1987,4.38
8,Miss Rumphius,Barbara Cooney,1982,4.44
9,Chrysanthemum,Kevin Henkes,1991,4.33


In [951]:
print("Which books does the Louisville library have out of the top 500 most popular physical Goodreads books?")

goodreadsp = goodreads.sort_values(by=["num_reviews", "num_ratings", "current_readers", "want_to_read"], ascending=False)
goodreadsp = goodreadsp.head(500)

mostpopular = pd.merge(loulib, goodreadsp, on=["title", "authors"], suffixes=("_lib", "_gr"), how="inner")
mostpopular[["title", "authors", "publication_date", "num_reviews", "num_ratings", "current_readers", "want_to_read"]]

Which books does the Louisville library have out of the top 500 most popular physical Goodreads books?


,title,authors,publication_date,num_reviews,num_ratings,current_readers,want_to_read
0,Atonement,Ian McEwan,2001,22228.0,515973.0,11200.0,365000.0
1,Life of Pi,Yann Martel,2001,52968.0,1610506.0,25600.0,765000.0
2,Matilda,Roald Dahl,1988,23073.0,931673.0,9297.0,318000.0
3,Coraline,Neil Gaiman,2002,36059.0,682081.0,10600.0,609000.0
4,Stardust,Neil Gaiman,1999,25436.0,448483.0,9064.0,373000.0
5,Stardust,Neil Gaiman,1999,25436.0,448480.0,9064.0,373000.0
6,Fight Club,Chuck Palahniuk,1996,21279.0,586225.0,9277.0,415000.0
7,The Guernsey Literary and Potato Peel Pie Society,Mary Ann Shaffer,2008,61826.0,747089.0,15400.0,382000.0
8,Little Bee,Chris Cleave,2008,22341.0,241639.0,4609.0,85500.0
9,11/22/63,Stephen King,2011,47249.0,533469.0,31000.0,25100.0


In [952]:
#writing more tables to csv to be used in tableau
goodreads = goodreads.reset_index()
loulib = loulib.reset_index()
loulib.to_csv("data/clean/loulib.csv", index=True)
goodreads.to_csv("data/clean/goodreads.csv", index=True)
goodreadst.to_csv("data/clean/goodreadst.csv", index=True)
goodreadsp.to_csv("data/clean/goodreadsp.csv", index=True)
#relational database Books, Popularity, Library Inventory
booksrel = pd.merge(loulib, goodreads, on=["title", "authors"], suffixes=("_lib", "_gr"), how = "outer")
booksrel = booksrel.drop(["isbn_lib", "Copies", "Versions", "isbn_gr", "rating_score", "num_ratings", "num_reviews", "current_readers", "want_to_read", "ItemType", "ItemCollection", "format"], axis=1)
booksrel.to_csv("data/clean/booksrel.csv", index=True)

poprel = pd.merge(loulib, goodreads, on=["title", "authors"], suffixes=("_lib", "_gr"), how = "outer")
poprel = poprel.drop(["isbn_lib", "ItemType", "ItemCollection", "isbn_gr", "Copies", "Versions"], axis=1)
poprel.to_csv("data/clean/poprel.csv", index=True)

libinvrel = pd.merge(loulib, goodreads, on=["title", "authors"], suffixes=("_lib", "_gr"), how = "outer")
libinvrel = libinvrel.drop(["isbn_lib", "isbn_gr", "format", "price", "rating_score", "num_ratings", "num_reviews", "current_readers", "want_to_read"], axis=1)
libinvrel.to_csv("data/clean/libinvrel.csv", index=True)

print("files created")

files created
